In [ ]:
import pickle
with open('/content/drive/MyDrive/Colab Notebooks/MDDdataset/symptom_sum_top16/train.pkl', 'rb') as f:
      raw_train_data = pickle.load(f)
with open('/content/drive/MyDrive/Colab Notebooks/MDDdataset/symptom_sum_top16/test.pkl', 'rb') as f:
      raw_test_data = pickle.load(f)
with open('/content/drive/MyDrive/Colab Notebooks/MDDdataset/symptom_sum_top16/val.pkl', 'rb') as f:
      raw_val_data = pickle.load(f)

In [ ]:
import pandas as pd
train_data=pd.DataFrame(raw_train_data)
test_data=pd.DataFrame(raw_test_data)
val_data=pd.DataFrame(raw_val_data)

In [ ]:
df_all = pd.concat([train_data, val_data], axis=0)
df_all = pd.concat([df_all, test_data], axis=0)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Enriched OCD keywords based on the Symptom Checklist-90 OCD Subscale
ocd_keywords = {
    "check and double-check": ["check and double-check", "repeatedly checking", "constantly verifying", "ensure everything is correct"],
    "do things very slowly": ["do things very slowly", "meticulously slow", "overly cautious", "extreme precision"],
    "mind going blank": ["mind going blank", "can't think clearly", "mental fog", "blank out"],
    "trouble remembering": ["trouble remembering", "forgetful", "can't recall", "memory issues"],
    "difficulty making decisions": ["difficulty making decisions", "indecisive", "can't decide", "hesitation"],
    "trouble concentrating": ["trouble concentrating", "easily distracted", "lack of focus", "can't concentrate"],
    "worried about sloppiness": ["worried about sloppiness", "fear of making mistakes", "perfectionist tendencies", "concerned about errors"],
    "feeling blocked": ["feeling blocked", "unable to progress", "stuck", "impeded"],
    "repeat the same actions": ["repeat the same actions", "compulsive behavior", "repetitive actions", "counting, washing"],
    "unwanted thoughts": ["unwanted thoughts", "intrusive thoughts", "obsessive thoughts", "can't get it out of my head"]
}

# Function to preprocess and score posts with a reduced TF-IDF
def preprocess_and_score(data):
    vectorizer = TfidfVectorizer(lowercase=True, stop_words='english', ngram_range=(1,2), max_features=5000)  # Limit features to control memory
    corpus = []
    for index, row in data.iterrows():
        if isinstance(row['selected_posts'], str):
            posts = eval(row['selected_posts'])
        else:
            posts = row['selected_posts']
        corpus.append(' '.join(posts))
    X = vectorizer.fit_transform(corpus)
    y = data['diseases'].apply(lambda x: 1 if 'ocd' in eval(str(x)) else 0)
    return X, y

X, y = preprocess_and_score(train_data)

# Simplified bootstrapping
n_bootstraps = 100  # Reduced number of bootstraps
bootstrap_f1 = []
bootstrap_sensitivities = []
bootstrap_specificities = []

for _ in range(n_bootstraps):
    sample_indices = np.random.choice(range(len(y)), size=len(y), replace=True)
    X_sample = X[sample_indices]
    y_sample = y.iloc[sample_indices]

    X_train, X_test, y_train, y_test = train_test_split(X_sample, y_sample, test_size=0.3, random_state=42)
    model = LogisticRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    f1 = f1_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm.ravel()
    sensitivity = tp / (tp + fn) if (tp + fn) != 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) != 0 else 0

    bootstrap_f1.append(f1)
    bootstrap_sensitivities.append(sensitivity)
    bootstrap_specificities.append(specificity)

# Calculate means and standard deviations
mean_f1 = np.mean(bootstrap_f1)
std_dev_f1 = np.std(bootstrap_f1)
mean_sensitivity = np.mean(bootstrap_sensitivities)
std_dev_sensitivity = np.std(bootstrap_sensitivities)
mean_specificity = np.mean(bootstrap_specificities)
std_dev_specificity = np.std(bootstrap_specificities)

# Print results
print(f"F1 Score: {mean_f1:.3f} ± {std_dev_f1:.3f}")
print(f"Sensitivity: {mean_sensitivity:.3f} ± {std_dev_sensitivity:.3f}")
print(f"Specificity: {mean_specificity:.3f} ± {std_dev_specificity:.3f}")



F1 Score: 0.171 ± 0.039
Sensitivity: 0.095 ± 0.024
Specificity: 1.000 ± 0.000


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, make_scorer, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, cross_val_score, cross_validate

# Enriched OCD keywords based on the Symptom Checklist-90 OCD Subscale
ocd_keywords = {
    "check and double-check": ["check and double-check", "repeatedly checking", "constantly verifying", "ensure everything is correct"],
    "do things very slowly": ["do things very slowly", "meticulously slow", "overly cautious", "extreme precision"],
    "mind going blank": ["mind going blank", "can't think clearly", "mental fog", "blank out"],
    "trouble remembering": ["trouble remembering", "forgetful", "can't recall", "memory issues"],
    "difficulty making decisions": ["difficulty making decisions", "indecisive", "can't decide", "hesitation"],
    "trouble concentrating": ["trouble concentrating", "easily distracted", "lack of focus", "can't concentrate"],
    "worried about sloppiness": ["worried about sloppiness", "fear of making mistakes", "perfectionist tendencies", "concerned about errors"],
    "feeling blocked": ["feeling blocked", "unable to progress", "stuck", "impeded"],
    "repeat the same actions": ["repeat the same actions", "compulsive behavior", "repetitive actions", "counting, washing"],
    "unwanted thoughts": ["unwanted thoughts", "intrusive thoughts", "obsessive thoughts", "can't get it out of my head"]
}

# Function to preprocess and score posts with a reduced TF-IDF
def preprocess_and_score(data):
    vectorizer = TfidfVectorizer(lowercase=True, stop_words='english', ngram_range=(1,2), max_features=5000)
    corpus = []
    for index, row in data.iterrows():
        if isinstance(row['selected_posts'], str):
            posts = eval(row['selected_posts'])
        else:
            posts = row['selected_posts']
        corpus.append(' '.join(posts))
    X = vectorizer.fit_transform(corpus)
    y = data['diseases'].apply(lambda x: 1 if 'ocd' in eval(str(x)) else 0)
    return X, y

X, y = preprocess_and_score(df_all)

# Function to calculate sensitivity and specificity
def sensitivity_score(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    return tp / (tp + fn) if (tp + fn) != 0 else 0

def specificity_score(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    return tn / (tn + fp) if (tn + fp) != 0 else 0

# Prepare the scorer functions
scoring = {'f1': 'f1', 'sensitivity': make_scorer(sensitivity_score), 'specificity': make_scorer(specificity_score)}

# Perform 5-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
model = LogisticRegression()
cv_results = cross_validate(model, X, y, cv=kf, scoring=scoring)

# Calculate mean and standard deviation for each metric
mean_f1 = np.mean(cv_results['test_f1'])
std_dev_f1 = np.std(cv_results['test_f1'])
mean_sensitivity = np.mean(cv_results['test_sensitivity'])
std_dev_sensitivity = np.std(cv_results['test_sensitivity'])
mean_specificity = np.mean(cv_results['test_specificity'])
std_dev_specificity = np.std(cv_results['test_specificity'])

# Print results
print(f"F1 Score: {mean_f1:.3f} ± {std_dev_f1:.3f}")
print(f"Sensitivity: {mean_sensitivity:.3f} ± {std_dev_sensitivity:.3f}")
print(f"Specificity: {mean_specificity:.3f} ± {std_dev_specificity:.3f}")


F1 Score: 0.163 ± 0.015
Sensitivity: 0.090 ± 0.010
Specificity: 1.000 ± 0.000
